In [10]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM

In [4]:
apple = pd.read_csv("../data/AAPL.csv", sep=",")

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.express as px

def create_rolling_features(data, window_size=2):
    data.sort_values('Date', inplace=True)
    data.set_index('Date', inplace=True)

    data['Rolling_Close_Mean'] = data['Close'].rolling(f'{window_size*30}D').mean()
    data['Rolling_Close_Std'] = data['Close'].rolling(f'{window_size*30}D').std()

    return data

# Charger les données (remplacez "votre_fichier.csv" par le chemin réel de votre fichier)
apple = pd.read_csv("../data/AAPL.csv", sep=",")
apple['Date'] = pd.to_datetime(apple['Date'])

# Créer les features avec la rolling window
apple = create_rolling_features(apple)

# Normaliser les données
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(apple[['Close', 'Rolling_Close_Mean', 'Rolling_Close_Std']])

# Préparer les données pour l'entraînement LSTM
X, y = [], []
for i in range(len(scaled_data)-30):  # Utiliser une fenêtre de 30 jours pour les prédictions futures
    X.append(scaled_data[i:i+30, :])
    y.append(scaled_data[i+30, 0])

X, y = np.array(X), np.array(y)

# Utiliser TimeSeriesSplit pour diviser les données
tscv = TimeSeriesSplit()

# Stocker les prédictions et les vraies valeurs dans des listes
predictions_list = []
actual_values_list = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Instancier le modèle LSTM
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Entraîner le modèle
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Faire des prédictions
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler.inverse_transform(np.concatenate((X_test[:, -1, 1:], y_pred_scaled), axis=1))[:, 0]

    # Stocker les prédictions et les vraies valeurs
    predictions_list.extend(y_pred)
    actual_values_list.extend(scaler.inverse_transform(np.concatenate((X_test[:, -1, 1:], y_test.reshape(-1, 1)), axis=1))[:, 0])

# Calculer la MAE finale
mae_final = mean_absolute_error(actual_values_list, predictions_list)
print(f'MAE Finale: {mae_final}')

# Créer un DataFrame pour les prédictions et les vraies valeurs
results_df = pd.DataFrame({
    'Date': apple.index[-len(predictions_list):],
    'Actual Values': actual_values_list,
    'Predictions': predictions_list
})

# Créer un graphique Plotly Express
fig = px.line(results_df, x='Date', y=['Actual Values', 'Predictions'],
              labels={'value': 'Valeur', 'variable': 'Type'},
              title='Prédictions vs Valeurs Réelles')

# Mettre en forme les couleurs
fig.update_traces(line=dict(color='blue'), selector=dict(name='Actual Values'))
fig.update_traces(line=dict(color='red'), selector=dict(name='Predictions'))

# Afficher le graphique
fig.show()


Epoch 1/10
15/15 [==============================] - 2s 17ms/step - loss: nan
Epoch 2/10
15/15 [==============================] - 0s 15ms/step - loss: nan
Epoch 3/10
15/15 [==============================] - 0s 16ms/step - loss: nan
Epoch 4/10
15/15 [==============================] - 0s 16ms/step - loss: nan
Epoch 5/10
15/15 [==============================] - 0s 16ms/step - loss: nan
Epoch 6/10
15/15 [==============================] - 0s 15ms/step - loss: nan
Epoch 7/10
15/15 [==============================] - 0s 17ms/step - loss: nan
Epoch 8/10
15/15 [==============================] - 0s 17ms/step - loss: nan
Epoch 9/10
15/15 [==============================] - 0s 18ms/step - loss: nan
Epoch 10/10
15/15 [==============================] - 0s 4ms/step
Epoch 1/10
29/29 [==============================] - 2s 14ms/step - loss: nan
Epoch 2/10
29/29 [==============================] - 0s 15ms/step - loss: nan
Epoch 3/10
29/29 [==============================] - 0s 13ms/step - loss: nan
Epoch 4/10
